<p>Department Of Computer Science

COMP432, COMPUTER SECURITY

Dr. Mohammad Alkhanafse

Section 2

</p>

<div align="center">

<img src="https://github.com/sondosaabed/Introducing-Generative-AI-with-AWS/assets/65151701/01485d19-c6d6-4072-99d7-178ea8ec4364" alt="Birzeit University Logo" height="170px">


# Iris eye Recognition

A Biometric Authentication system using 

</div>

<b>Prepared by:</b> Sondos Aabed

<b>Studnet ID:</b> 1190652

<hr>

## Abstract

<hr>

## Table of Contents

## List of Figures

## List of Tables

<hr>

## Introduction

### Aim and Objectives

### About Dataset used

### Methodogly 

<hr>

## Theory

<hr>

## Software Listing

<hr>

## Implemetation

<hr>

### Dataset Analysis

#### Loading Dataset

In [ ]:
def load_dataset(path):
    """
    
    """

#### Expolring Dataset

In [ ]:
def explore_dataset(df):
    """
    
    """

#### Preparing Dataset

##### Images preparing

In [ ]:
def preprocess_images(df):
    """
    
    """

##### labels preparing:

In [ ]:
def preprocess_labels(df):
    """
    
    """

In [ ]:
def prepare_dataset(df):
    """
    
    """
    images = preprocess_images(df)
    labels = preprocess_labels(df)
    return images, labels

<hr>

### Data modeling

#### Model Archeticticure:

##### Base model (Feature Extracture)

In [ ]:
def feature_extraction():
    """
    
    """

#### Recognition model

In [ ]:
def reconition_model(featurres):
    """
    
    """

#### Model Training and testing:

#### Model Evaluaion metrics:

<hr>

### User Experinece and GUI

#### Image Accuisition.

#### Image Preparing for Recognition

In [ ]:
def input_image_preprocesisng(image):
    """
    
    """

#### Model Inference

<hr>

## Conclusion

## Refrences